In [ ]:
# MSc Project: Biometric Security - Baseline Model
## Author: Stella Williams
## Date: 04.10.2025

#---

## 1. Imports and Setup
## 2. Dataset Loading (LFW)
## 3. Load Pretrained ResNet18 and Freeze Base
## 4. Add Custom Classification Head
## 5. Train for a Few Epochs
## 6. Save the Model
## 7. Evaluate on Clean Samples
## 8. Set Up for FGSM and Defence Experiments (Future Work)

# 1. Imports and Setup

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import datasets, models
from torch.utils.data import DataLoader
import os
import time
import matplotlib.pyplot as plt


# 2. Dataset Loading

# Adjust path if needed
lfw_path = "../Datasets/lfw-dataset"

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # for ResNet input size
    transforms.ToTensor(),
])



lfw_dataset = datasets.ImageFolder(root=lfw_path, transform=transform)
lfw_loader = DataLoader(lfw_dataset, batch_size=16, shuffle=True)


# Check class count
num_classes = len(lfw_dataset.classes)
print("Number of classes:", num_classes)

# Resnet stuff for classification

import torch
import torch.nn as nn
from torchvision import models

# Detect device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

from torchvision.models import resnet18, ResNet18_Weights

model = resnet18(weights=ResNet18_Weights.DEFAULT)

# Freeze early layers (optional)
for param in model.parameters():
    param.requires_grad = False

# Replace final classification layer to match LFW classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(lfw_dataset.classes))

# Move model to device
model = model.to(device)

# Set Up Loss and Optimiser

import torch.optim as optim
import time

# Set model to training mode
model.train()

# Define loss and optimiser
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Limit training for prototyping
num_batches = 20
start_time = time.time()

for batch_idx, (images, labels) in enumerate(lfw_loader):
    if batch_idx >= num_batches:
        break
    
    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    
    print(f"Batch {batch_idx+1}/{num_batches} - Loss: {loss.item():.4f}")

print(f"\nFinished training {num_batches} batches in {(time.time() - start_time):.2f} seconds.")

torch.save(model.state_dict(), "resnet_lfw_prelim.pth")

# --- FGSM Attack Function ---
def fgsm_attack(image, epsilon, data_grad):
    sign_data_grad = data_grad.sign()
    perturbed_image = image + epsilon * sign_data_grad
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    return perturbed_image

# selecting image for ATTACKKKK 
model.eval()
attempts = 0
max_attempts = 30  # Try 30 images max

for images, labels in lfw_loader:
    for i in range(images.shape[0]):
        image = images[i].unsqueeze(0)
        label = torch.tensor([labels[i]])

        image.requires_grad = True
        output = model(image)
        init_pred = output.max(1, keepdim=True)[1]

        if init_pred.item() == label.item():
            print(f"✅ Found correctly classified image at attempt {attempts+1}")
            loss = criterion(output, label)
            model.zero_grad()
            loss.backward()

            epsilon = 0.1
            data_grad = image.grad.data
            perturbed_image = fgsm_attack(image, epsilon, data_grad)

            output_adv = model(perturbed_image)
            adv_pred = output_adv.max(1, keepdim=True)[1]

            print(f"Original prediction: {lfw_dataset.classes[init_pred.item()]}")
            print(f"Adversarial prediction: {lfw_dataset.classes[adv_pred.item()]}")
            print(f"Clean confidence: {torch.softmax(output, dim=1)[0][init_pred].item():.4f}")
            print(f"Adversarial confidence: {torch.softmax(output_adv, dim=1)[0][adv_pred].item():.4f}")

            # Visualise
            import matplotlib.pyplot as plt
            import torchvision.transforms.functional as TF

            plt.figure(figsize=(8, 4))
            plt.subplot(1, 2, 1)
            plt.title("Original")
            plt.imshow(TF.to_pil_image(image.squeeze().detach()))
            plt.axis('off')

            plt.subplot(1, 2, 2)
            plt.title("Adversarial")
            plt.imshow(TF.to_pil_image(perturbed_image.squeeze().detach()))
            plt.axis('off')
            plt.suptitle(f"Epsilon: {epsilon}")
            plt.show()

            break  # Stop after first success
        attempts += 1
        if attempts >= max_attempts:
            print("❌ No correctly predicted image found after 30 tries.")
            break
    else:
        continue
    break
    
    # Run the same FGSM loop with different epsilon values:

epsilons = [0.01, 0.05, 0.1, 0.15, 0.2]


# Plot how adversarial success rate changes.


Number of classes: 5749
Using device: cpu
Batch 1/20 - Loss: 8.5900
Batch 2/20 - Loss: 8.7848
Batch 3/20 - Loss: 8.5426
Batch 4/20 - Loss: 8.8416
Batch 5/20 - Loss: 8.8625
Batch 6/20 - Loss: 8.3139
Batch 7/20 - Loss: 8.8097
Batch 8/20 - Loss: 8.9172
Batch 9/20 - Loss: 8.8381
Batch 10/20 - Loss: 9.3917
Batch 11/20 - Loss: 9.3120
Batch 12/20 - Loss: 9.2534
Batch 13/20 - Loss: 10.6001
Batch 14/20 - Loss: 10.1084
Batch 15/20 - Loss: 10.6524
Batch 16/20 - Loss: 10.1895
Batch 17/20 - Loss: 11.4078
Batch 18/20 - Loss: 9.7995
Batch 19/20 - Loss: 10.1983
Batch 20/20 - Loss: 10.3403

Finished training 20 batches in 8.90 seconds.
✅ Found correctly classified image at attempt 20
Original prediction: George_W_Bush
Adversarial prediction: Donald_Rumsfeld
Clean confidence: 0.0530
Adversarial confidence: 0.0317
